In [3]:
import pandas as pd    
  
# 读取Excel文件    
file_path = '专题统计_实际控制人持股变动.xlsx'  # 请替换为你的Excel文件的路径    
df = pd.read_excel(file_path, engine='openpyxl', dtype={'证券代码': str})    
  
# 创建一个包含要排除的所有关键字的列表  
exclude_keywords = ['国有', '国务院', '人民政府', '中共']  
  
# 使用str.contains()函数检查“实际控制人名称”列中的值是否包含关键字列表中的任何一个关键字，并使用'~'操作符取反，从而筛选出不包含这些关键字的行  
filtered_df = df[~df['实际控制人名称'].str.contains('|'.join(exclude_keywords), na=False)]    
  
# 提取证券代码列并转换为列表    
stock_codes = filtered_df['证券代码'].tolist()    
  
# print(stock_codes)
# 使用列表推导式筛选出以"00"开头的股票代码  
stock_codes_60_prefix = [code for code in stock_codes if code.startswith("60")]  
print(stock_codes_60_prefix)

['600031', '600051', '600152', '600158', '600172', '600180', '600290', '600293', '600370', '600405', '600478', '600496', '600557', '600588', '600682', '600770', '600818', '600360', '600545', '600481', '600598', '600094', '600110', '600118', '600143', '600161', '600165', '600171', '600195', '600227', '600297', '600327', '600576', '600580', '600596', '600647', '600696', '600703', '600745', '600753', '600766', '600857', '600363', '600750', '601880', '600694', '600130', '600070', '600079', '600095', '600112', '600177', '600208', '600210', '600216', '600257', '600276', '600308', '600337', '600340', '600351', '600353', '600380', '600438', '600439', '600477', '600507', '600522', '600571', '600577', '600594', '600728', '600735', '600785', '600803', '600998', '600288', '600654', '600699', '600681', '600246', '600339', '600109', '600371', '600265', '600255', '600089', '600277', '600280', '600331', '600355', '600372', '600381', '600491', '600503', '600506', '600603', '600710', '600711', '600749',

In [4]:
stock_codes = []
# 股票代码列表    
for i in stock_codes_60_prefix:
    stock_codes.append('sh.'+i)
stock_codes

['sh.600031',
 'sh.600051',
 'sh.600152',
 'sh.600158',
 'sh.600172',
 'sh.600180',
 'sh.600290',
 'sh.600293',
 'sh.600370',
 'sh.600405',
 'sh.600478',
 'sh.600496',
 'sh.600557',
 'sh.600588',
 'sh.600682',
 'sh.600770',
 'sh.600818',
 'sh.600360',
 'sh.600545',
 'sh.600481',
 'sh.600598',
 'sh.600094',
 'sh.600110',
 'sh.600118',
 'sh.600143',
 'sh.600161',
 'sh.600165',
 'sh.600171',
 'sh.600195',
 'sh.600227',
 'sh.600297',
 'sh.600327',
 'sh.600576',
 'sh.600580',
 'sh.600596',
 'sh.600647',
 'sh.600696',
 'sh.600703',
 'sh.600745',
 'sh.600753',
 'sh.600766',
 'sh.600857',
 'sh.600363',
 'sh.600750',
 'sh.601880',
 'sh.600694',
 'sh.600130',
 'sh.600070',
 'sh.600079',
 'sh.600095',
 'sh.600112',
 'sh.600177',
 'sh.600208',
 'sh.600210',
 'sh.600216',
 'sh.600257',
 'sh.600276',
 'sh.600308',
 'sh.600337',
 'sh.600340',
 'sh.600351',
 'sh.600353',
 'sh.600380',
 'sh.600438',
 'sh.600439',
 'sh.600477',
 'sh.600507',
 'sh.600522',
 'sh.600571',
 'sh.600577',
 'sh.600594',
 'sh.6

In [5]:
import baostock as bs    
import pandas as pd    
import os  
  
# 登陆系统    
lg = bs.login()    
# 显示登陆返回信息    
print('login respond error_code:' + lg.error_code)    
print('login respond  error_msg:' + lg.error_msg)    
    

  
# 指定存放Excel文件的文件夹路径  
folder_path = "非国企沪市的季频现金流数据"  # 替换为你的文件夹路径  
  
# 如果文件夹不存在，创建它  
if not os.path.exists(folder_path):  
    os.makedirs(folder_path)  
  
# 遍历每个股票    
for code in stock_codes:    
    print(f"Processing {code}")  
    all_data_list = []    
    # 遍历每一年的四个季度    
    for year in range(2007, 2024):  # 2024用于包括2023年的数据    
        for quarter in range(1, 5):  # 一年有四个季度    
            print(f"Processing {year} quarter {quarter}")    
            # 季频现金流量
            cash_flow_list = []    
            rs_cash_flow = bs.query_cash_flow_data(code=code, year=year, quarter=quarter)    
            while (rs_cash_flow.error_code == '0') & rs_cash_flow.next():    
                cash_flow_list.append(rs_cash_flow.get_row_data())    
            # 将数据添加到列表中    
            result_cash_flow = pd.DataFrame(cash_flow_list, columns=rs_cash_flow.fields)    
            result_cash_flow['code'] = code    
            result_cash_flow['year'] = year    
            result_cash_flow['quarter'] = quarter    
            all_data_list.append(result_cash_flow)    
              
    # 合并该股票的所有数据    
    all_data = pd.concat(all_data_list, ignore_index=True)    
    # 将该股票的数据输出到单独的Excel文件中  
    excel_file_path = os.path.join(folder_path, f"{code}.xlsx") # 创建该股票的Excel文件路径  
    all_data.to_excel(excel_file_path, index=False) # 将数据保存到该Excel文件中  
    print(f"Data for {code} saved to {excel_file_path}\n") # 输出保存信息  
        
# 登出系统    
bs.logout()

login success!
login respond error_code:0
login respond  error_msg:success
Processing sh.600031
Processing 2007 quarter 1
Processing 2007 quarter 2
Processing 2007 quarter 3
Processing 2007 quarter 4
Processing 2008 quarter 1
Processing 2008 quarter 2
Processing 2008 quarter 3
Processing 2008 quarter 4
Processing 2009 quarter 1
Processing 2009 quarter 2
Processing 2009 quarter 3
Processing 2009 quarter 4
Processing 2010 quarter 1
Processing 2010 quarter 2
Processing 2010 quarter 3
Processing 2010 quarter 4
Processing 2011 quarter 1
Processing 2011 quarter 2
Processing 2011 quarter 3
Processing 2011 quarter 4
Processing 2012 quarter 1
Processing 2012 quarter 2
Processing 2012 quarter 3
Processing 2012 quarter 4
Processing 2013 quarter 1
Processing 2013 quarter 2
Processing 2013 quarter 3
Processing 2013 quarter 4
Processing 2014 quarter 1
Processing 2014 quarter 2
Processing 2014 quarter 3
Processing 2014 quarter 4
Processing 2015 quarter 1
Processing 2015 quarter 2
Processing 2015 quar